# Sudoku

I recently started solving some sudoku puzzles for fun,
and the thought occured to me: *Why not try to write a sudoku solver-generator in Python?*

## No backtracking, please...

A quick google search for how to solve sudokus programmatically
will give you many *backtracking* solutions. While *backtracking*
is a great way to solve sudokus, I don't like it.
Backtracking essentially is just *guessing random numbers until the sudok is solved*.
Backtracking not only lacks elegance,
but, from an *algorithmic* perspective, it's not very efficient and doesn't scale well.

I wanted to solve a sudoku differently. I wanted to come up with an algorithm
that more closely matched how one might solve a sudoku in real life: by using
logic!

## Graph processing approach

When I solve a sudoku I am constantly thinking
about how all of the cells are interconnected.
They can be connected to each other either by
being in the same *row*, *column*, or *box*.
These complex connections are best modeled as a *graph*.

The purpose of this tutorial is to see if we can use *graph-processing*
to solve a sudoku.

```{note}
This tutorial is limited to 9x9 sudokus.
It wouldn't be too much work to generalize
this solution to other sizes of sudokus, but I just don't 
feel like it...
```


## Sudoku data structure

As mentioned previously, we will model our sudoku as a *graph*.
We will use [networkx](https://networkx.org/documentation/stable/index.html)
for its wonderful *graph* functionalities.

We represent each *cell* of the sudoku as a *node*.
The nodes be equal to the index of each cell. E.g `(0,0)`
is the top-left cell, and `(8,8)` is the bottom-right cell.

We represent the *connections* between nodes by *edges*.
Cells can be connected by being in the same *row*, *edge*, or *box*.

In [ ]:
import itertools
import networkx as nx
import matplotlib.pyplot as plt
from typing import List, Dict, Union, Callable

Sudoku = nx.MultiGraph

def gen_sudoku()->Sudoku:
    n=3
    dim = n*n
    sudoku = Sudoku()
    sudoku.graph["possible_values"] = set(range(1, 10))
    
    # Create a node for each cell in the sudoku
    for row, col in itertools.product(range(dim), range(dim)):
        # Each node is the row-column, and has a 'value'
        # We set the value to '0' initially
        sudoku.add_node((row,col), value=0)
        
    # Create the edges for each combination of nodes
    sudoku.graph["connections"] = ["row", "column", "box"]
    for n1, n2 in itertools.combinations(sudoku.nodes, r=2):
        n1_row, n1_col = n1
        n2_row, n2_col = n2
        n1_box = [i // n for i in [n1_row, n1_col]]
        n2_box = [i // n for i in [n2_row, n2_col]]
        
        if n1_row == n2_row:
            sudoku.add_edge(n1, n2, connection="row")
        if n1_col == n2_col:
            sudoku.add_edge(n1, n2, connection="column")
        if n1_box == n2_box:
            sudoku.add_edge(n1, n2, connection="box")
        
    return sudoku

sudoku = gen_sudoku()

## Visualizing the sudoku

Now that we have a sudoku data structure,
let's visualize it.

In [ ]:
def plot_sudoku(sudoku)->None:
    # - node (0,0) is at pos (0,8)
    # - node (0,1) is at pos (1,8)
    # - node (8,8) is at pos(8,0)
    pos = {n: (n[1], 8 - n[0]) for n in sudoku.nodes}
    labels = {n:sudoku.nodes[n]["value"] for n in sudoku.nodes}
    nx.draw(
        sudoku, 
        pos=pos, 
        labels=labels, 
        with_labels=True, 
        font_color="black", 
        node_color=[v for v in labels.values()],
        cmap=plt.cm.Set3
    )
    low = -0.5
    high = 8.5
    lines = [low, 2.5, 5.5, high]
    for l in lines:
        plt.plot([l,l], [low, high], color="g")
        plt.plot([low, high], [l,l], color="g")
    plt.show()
    
plot_sudoku(sudoku)

Remember that the whole point of using a *graph data structure*
was to visualize *connections* between the nodes.

In [ ]:
SudokuConnections = dict[str, nx.Graph]

def get_sudoku_connections(sudoku:Sudoku)->SudokuConnections:
    all_nodes = list(sudoku.nodes(data=True))
    all_edges = [(u, v, c) for (u, v, c) in sudoku.edges(data="connection")]
    
    sudoku_connections: SudokuConnections = {}
    
    for conn in sudoku.graph["connections"]:
        sc = nx.Graph()
        sc.add_nodes_from(all_nodes)
        edges_to_keep = [(u, v) for u, v, c in all_edges if c == conn]
        sc.add_edges_from(edges_to_keep)
        sudoku_connections[conn] = sc
        
    sc_all = nx.Graph()
    sc_all.add_nodes_from(all_nodes)
    sc_all.add_edges_from([u, v] for u, v, c in all_edges)
    sudoku_connections["all"] = sc_all
    
    return sudoku_connections
    

def plot_connections(sudoku_connections):
    for conn, sc in sudoku_connections.items():
        plt.title(conn)
        plot_sudoku(sc)
    
    
sudoku_connections = get_sudoku_connections(sudoku)
plot_connections(sudoku_connections)

## Creating a *solver*

First, we need a way to tell if a sudoku is  actually solved.

In [ ]:
def is_sudoku_solved(sudoku:Sudoku)->bool:
    # a sudoku is unsolved if there are any zeros
    if any([v == 0 for n, v in sudoku.nodes(data="value")]):
        return False
    # a sudoku is unsolved if any 'connections' don't have all numbers 1-9
    sudoku_connections = get_sudoku_connections(sudoku)
    for conn, sc in sudoku_connections.items():
        for cc in nx.connected_components(sc):
            values = set(sudoku.nodes[n]["value"] for n in cc)
            if len(values) != len(sudoku.graph["possible_values"]):
                return False
    # if we made it this far, then sudoku is solved
    return True

is_sudoku_solved(sudoku)

A sudoku solver will be a function which takes a sudoku
as input, and returns a *solved* sudoku.
Our solver function also takes as input *rules*.
Each rule is itself a function which takes a sudoku, and a *node*
to solve. If the node can be solved, then it returns a value for the node.
If it cannot be solved, it returns `None`.

In [ ]:
def solve_sudoku(sudoku, *rules)->Sudoku:
    solved_sudoku = sudoku.copy()
    
    def do_solving():
        nodes_to_update = {}
        for rule in rules:
            rule_nodes_to_update = rule(solved_sudoku)
            nodes_to_update.update(rule_nodes_to_update)
        if nodes_to_update:
            for n, val in nodes_to_update.items():
                solved_sudoku.nodes[n]["value"] = val
            return    
        else:
            raise RuntimeError("Cannot solve sudoku", solved_sudoku)
            
    while not is_sudoku_solved(solved_sudoku):
        do_solving()
    return solved_sudoku
        

def try_solve_sudoku(sudoku, *rules):
    try:
        return solve_sudoku(sudoku, *rules)
    except RuntimeError as exc:
        msg, partially_solved_sudoku = exc.args
        return partially_solved_sudoku

In [ ]:
def ascending_values_rule(sudoku):
    nodes_to_update = {}
    for n in sudoku.nodes:
        r, c = n
        br = r // 3
        val = (r*3 + c + 1 + br) % 9 or 9
        nodes_to_update[n] = val
    return nodes_to_update

plot_sudoku(solve_sudoku(sudoku, ascending_values_rule))

In [ ]:
def gen_sudoku_with_values(values):
    sudoku = gen_sudoku()
    for i, row in enumerate(values):
        for j, val in enumerate(row):
            sudoku.nodes[(i,j)]["value"] = val
    return sudoku
            
easy_sudoku = gen_sudoku_with_values([
    [4,0,0,2,0,5,8,6,0],
    [0,0,0,0,0,8,4,9,3],
    [6,0,0,0,0,0,0,2,7],
    [1,0,0,0,6,9,0,5,0],
    [3,0,0,0,8,0,0,0,9],
    [0,5,0,4,3,0,0,0,2],
    [8,6,0,0,0,0,0,0,4],
    [7,2,1,8,0,0,0,0,0],
    [0,9,4,1,0,3,0,0,6]
])
plot_sudoku(easy_sudoku)

In [ ]:
def get_sudoku_impossible_values(sudoku:Sudoku):
    sudoku_impossibilities = {n:set() for n in sudoku.nodes}
    possible_values = sudoku.graph["possible_values"].copy()
    sudoku_connections = get_sudoku_connections(sudoku)
    
    for node, value in sudoku.nodes(data="value"):
        if value == 0:
            sc_all = sudoku_connections["all"]
            values_of_all_neighobrs = set([sudoku.nodes[n]["value"] for n in sc_all.neighbors(node)])
            values_of_all_neighobrs.discard(0)
            sudoku_impossibilities[node] = values_of_all_neighobrs
        else:
            all_other_values = possible_values - {value}
            sudoku_impossibilities[node] = all_other_values
            
    return sudoku_impossibilities

def solve_by_deducing_impossible_values(sudoku):
    impossible_values = get_sudoku_impossible_values(sudoku)
    possible_values = sudoku.graph["possible_values"]
    updates = {}
    for node, values in impossible_values.items():
        if sudoku.nodes[node]["value"] == 0 and len(values) == 8:
            value = list((possible_values - values))[0]
            updates[node] = value
    return updates
            
plot_sudoku(try_solve_sudoku(easy_sudoku, solve_by_deducing_impossible_values))

In [ ]:
%timeit try_solve_sudoku(easy_sudoku, solve_by_deducing_impossible_values)

In [ ]:
medium_sudoku = gen_sudoku_with_values([
    [9,0,0,3,0,0,0,0,0],
    [0,0,1,0,4,0,0,5,0],
    [8,0,6,0,0,1,0,4,0],
    [3,0,0,0,0,0,2,0,0],
    [1,0,5,4,3,2,7,0,9],
    [0,0,7,0,0,0,0,0,4],
    [0,1,0,5,0,0,4,0,6],
    [0,7,0,0,2,0,1,0,0],
    [0,0,0,0,0,7,0,0,8],
])
plot_sudoku(medium_sudoku)
medium_sudoku_solved = try_solve_sudoku(medium_sudoku, solve_by_deducing_impossible_values)
plot_sudoku(medium_sudoku_solved)
is_sudoku_solved(medium_sudoku_solved)

In [ ]:
def deduce_value_from_impossibilities(sudoku):
    possible_values = sudoku.graph["possible_values"]
    impossible_values = get_sudoku_impossible_values(sudoku)
    sudoku_connections = get_sudoku_connections(sudoku)
    updates = {}
    for node, value in sudoku.nodes(data="value"):
        if value == 0:
            for conn, sc in sudoku_connections.items():
                neighbors = sc.neighbors(node)
                imp_vals_of_neighbors = possible_values.copy()
                imp_vals_of_neighbors.intersection_update(
                    *[impossible_values[n] for n in neighbors]
                )
                if len(imp_vals_of_neighbors) == 1:
                    updates[node] = list(imp_vals_of_neighbors)[0]
    print(updates)
    return updates
            
plot_sudoku(try_solve_sudoku(medium_sudoku, deduce_value_from_impossibilities))

In [ ]:
hard_sudoku = gen_sudoku_with_values([
    [2,0,0,0,6,0,7,3,0],
    [0,0,9,0,2,3,0,8,6],
    [0,0,0,5,0,8,1,0,0],
    [9,5,0,0,0,0,0,1,0],
    [8,0,6,0,3,0,5,0,9],
    [0,4,0,0,0,0,0,7,8],
    [0,0,2,3,0,5,0,0,0],
    [4,1,0,2,9,0,3,0,0],
    [0,6,5,0,8,0,0,0,1],
])
plot_sudoku(hard_sudoku)
hard_sudoku_solved = try_solve_sudoku(hard_sudoku, deduce_value_from_impossibilities)
plot_sudoku(hard_sudoku_solved)
is_sudoku_solved(hard_sudoku_solved)

In [ ]:
evil_sudoku = gen_sudoku_with_values([
    [0,1,0,0,0,8,0,0,3],
    [0,0,0,9,0,1,0,2,8],
    [0,0,4,0,0,0,0,0,0],
    [0,0,1,0,0,0,0,3,4],
    [0,3,0,0,6,0,0,9,0],
    [2,5,0,0,0,0,7,0,0],
    [0,0,0,0,0,0,3,0,0],
    [6,4,0,5,0,9,0,0,0],
    [9,0,0,8,0,0,0,1,0],
])
plot_sudoku(evil_sudoku)
evil_sudoku_solved = try_solve_sudoku(
    evil_sudoku,
    solve_by_deducing_impossible_values,
    deduce_value_from_impossibilities
)
plot_sudoku(evil_sudoku_solved)
print(is_sudoku_solved(evil_sudoku_solved))

In [ ]:
def get_successor_nodes(sudoku, *source_nodes):
    successor_nodes = set()
    for sn in source_nodes:
        bfs_succ = dict(nx.bfs_successors(sudoku, sn, depth_limit=1))
        for u, vs in bfs_succ.items():
            [successor_nodes.add(v) for v in vs]
        
    return list(successor_nodes)

def plot_sub_sudoku(sudoku, *source_nodes):
    successor_nodes = get_successor_nodes(sudoku, *source_nodes)
    plot_sudoku(sudoku.subgraph(successor_nodes))
    
plot_sub_sudoku(sudoku, (0,0))

In [ ]:
def get_nodes_with_value(sudoku, value):
    nodes = []
    for n in sudoku.nodes:
        if value == sudoku.nodes[n]["value"]:
            nodes.append(n)
    return nodes

nodes_with_4 = get_nodes_with_value(easy_sudoku, 4)
nodes_with_4

In [ ]:
plot_sub_sudoku(easy_sudoku, *nodes_with_4)

In [ ]:
possible_values = set(range(1,10))

def is_sudoku_solved(sudoku):
    return all(sudoku.nodes[n]["value"] != 0 for n in sudoku.nodes)
    
def get_sudoku_impossible_values(sudoku):
    sudoku_impossibilities = {n:set() for n in sudoku.nodes}
    for value in possible_values:
        nodes_with_value = get_nodes_with_value(sudoku, value)
        for n in nodes_with_value:
            all_other_vals = possible_values - {value}
            sudoku_impossibilities[n] = all_other_vals
        successor_nodes = get_successor_nodes(sudoku, *nodes_with_value)
        for sn in successor_nodes:
            sudoku_impossibilities[sn].add(value)
    return sudoku_impossibilities

def solve(sudoku):
    new_sudoku = sudoku.copy()
                           
    def update_new_sudoku_values():
        sudoku_impossibilities = get_sudoku_impossible_values(new_sudoku)
        for n in new_sudoku.nodes:
            impossible_values = sudoku_impossibilities[n]
            if new_sudoku.nodes[n]["value"] == 0:
                if len(impossible_values) == 8:
                    new_sudoku.nodes[n]["value"] = list(possible_values - impossible_values)[0]
                    return
        else:
            raise RuntimeError("Cannot solve", new_sudoku)
                    
    while not is_sudoku_solved(new_sudoku):
        update_new_sudoku_values()
        
    return new_sudoku

def try_solve(sudoku):
    try:
        solve(sudoku)
    except Exception as exc:
        msg, unsolved_sudoku = exc.args
        return unsolved_sudoku

plot_sudoku(solve(easy_sudoku))

In [ ]:
impossible_values = get_sudoku_impossible_values(medium_sudoku_solved)

impossible_values_67 = impossible_values[(8,7)]
possible_values_67 = possible_values - impossible_values_67

successors_67 = get_successor_nodes(medium_sudoku_solved, (8,7))

successor_common_impossible = possible_values.copy()

for s in successors_67:
    print(s, impossible_values[s])
    
print(possible_values.intersection(*all_impossible_values_of_successors))
print(possible_values_67)
print(successor_common_impossible)